In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

panel = pd.read_pickle('../Data/returns_chars_panel.pkl') 
macro = pd.read_pickle('../Data/macro_timeseries.pkl')

In [5]:
# combine micro and macro data
df = pd.merge(panel,macro,on='date',how='left',suffixes=['','_macro']) 

# features + targets 
X = df.drop(columns=['ret','excess_ret','rfree','permno','date']) # everything except return info and IDs
y = df['excess_ret'] 

# Split Train Test 

In [19]:
# make 20 years of training data
date = df['date']
training = (date <= '1977-03') # selects 
X_train, y_train = X.loc[training].values, y.loc[training].values 

# make 10 years of validation data
validation = (date > '1977-03') & (date <= '1987-03') 
X_val, y_val = X.loc[validation].values, y.loc[validation].values 

# make test data
test = (date > '1987-03') 
X_test, y_test = X.loc[test].values, y.loc[test].values 

# Finding baseline model for Neural Network 

## 2 layers 

In [57]:
import tensorflow as tf
tf.random.set_seed(999)
from tensorflow.keras.regularizers import L2

def nn1(lamda):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='linear', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal')
    ])
    return model 

def nn2(lamda):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(16, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='linear', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal')
    ])
    return model 


def nn3(lamda):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(16, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(8, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='linear', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal')
    ])
    return model 

def nn4(lamda):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(16, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(8, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='linear', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal')
    ])
    return model 

def nn5(lamda):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(16, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(8, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(2, activation='relu', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal'), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='linear', kernel_regularizer = L2(lamda), kernel_initializer = 'he_normal')
    ])
    return model 

In [58]:
from tensorflow.keras.callbacks import EarlyStopping

# hypers
batch_size = 512
learning_rate = 1e-4
lamda = 1e-5
patience = 2

model = nn2(lamda)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    batch_size=batch_size, 
                    verbose=True,
                    validation_data = (X_val, y_val),
                    # Defined the callback to see if the validation result actually improves 
                    callbacks = [EarlyStopping(patience = patience, restore_best_weights=True)])

fig, ax = plt.subplots(figsize=(10,6))
pd.Series(history.history['mse']).plot(ax=ax, label='training mse')
pd.Series(history.history['val_mse']).plot(ax=ax, label='validation mse')
ax.legend()

ImportError: cannot import name 'modelling' from 'nn_models' (/Users/dhammatornriewcharoon/Documents/BD2_coursework/Code/nn_models.py)